In [50]:
library(geojsonio)
library(tidyverse)
library(dplyr)
library("knitr")
library(units)
library(sf)

In [51]:
# Read in the data
census_index_crime <- st_read("census_index_crime.geojson") |>
    mutate(across(c(immigrant_prop, education_prop, mean_land_value,income), as.double))|>
    na.omit()|>
    glimpse()



Reading layer `census_index_crime' from data source 
  `C:\Users\Kaiyan Zhang\Desktop\econ326\census_index_crime.geojson' 
  using driver `GeoJSON'
Simple feature collection with 993 features and 18 fields
Geometry type: MULTIPOLYGON
Dimension:     XY
Bounding box:  xmin: -123.2242 ymin: 49.19853 xmax: -123.0229 ymax: 49.31408
Geodetic CRS:  WGS 84


Warning message:
"There were 4 warnings in `stopifnot()`.
The first warning was:
ℹ In argument: `across(...)`.
Caused by warning:
! NAs introduced by coercion
ℹ Run `dplyr::last_dplyr_warnings()` to see the 3 remaining warnings."


Rows: 989
Columns: 19
$ name                 <chr> "59150307", "59150308", "59150309", "59150310", "…
$ households           <int> 254, 203, 269, 283, 181, 203, 402, 170, 333, 174,…
$ dwellings            <int> 273, 223, 299, 310, 199, 211, 418, 181, 356, 189,…
$ population           <int> 632, 501, 745, 536, 532, 562, 1088, 556, 959, 506…
$ population_density   <dbl> 2110.9, 4575.3, 6663.7, 4895.0, 6567.9, 6445.0, 6…
$ region               <chr> "Vancouver", "Vancouver", "Vancouver", "Vancouver…
$ area_sq_km           <dbl> 0.29926, 0.10956, 0.11190, 0.10940, 0.08094, 0.08…
$ age                  <int> 630, 500, 745, 540, 530, 560, 1085, 555, 960, 510…
$ income               <dbl> 0.90965, 0.93611, 0.84736, 0.45184, 0.82517, 0.74…
$ immigrant_prop       <dbl> 0.2610759, 0.2894212, 0.3557047, 0.2891791, 0.404…
$ education_prop       <dbl> 0.3085443, 0.2794411, 0.3154362, 0.1865672, 0.253…
$ school_index         <dbl> 1.074675, 1.252273, 1.426336, 2.441091, 1.543095,…
$ light_index     

In [55]:
# Test by running a model including all control
reg_simple <- lm(crime_rate ~ school_index + light_index + bus_index + cultural_space_index + immigrant_prop + education_prop, data = census_index_crime)
    
summary(reg_simple)


Call:
lm(formula = crime_rate ~ school_index + light_index + bus_index + 
    cultural_space_index + immigrant_prop + education_prop, data = census_index_crime)

Residuals:
    Min      1Q  Median      3Q     Max 
-1745.0  -290.4   -97.3   138.3 13110.7 

Coefficients:
                     Estimate Std. Error t value Pr(>|t|)    
(Intercept)           -579.87     164.98  -3.515  0.00046 ***
school_index          -297.28      36.33  -8.183 8.52e-16 ***
light_index            345.00      45.86   7.523 1.21e-13 ***
bus_index               43.89      51.23   0.857  0.39173    
cultural_space_index   226.95      40.93   5.544 3.79e-08 ***
immigrant_prop        -148.58     197.52  -0.752  0.45208    
education_prop        -171.98     197.14  -0.872  0.38323    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 732.5 on 982 degrees of freedom
Multiple R-squared:  0.2477,	Adjusted R-squared:  0.2431 
F-statistic: 53.89 on 6 and 982 DF,  p-value: < 2.

In [56]:
# Predict y_hat
y_hat <- predict(reg_simple)

length(y_hat)

[1] 989

In [57]:
# Carry out white's test
white <- lm(census_index_crime$crime_rate - y_hat ~ y_hat + I(y_hat^2))|>
    summary()

white


Call:
lm(formula = census_index_crime$crime_rate - y_hat ~ y_hat + 
    I(y_hat^2))

Residuals:
    Min      1Q  Median      3Q     Max 
-3103.4  -160.8   -48.6    63.1 12032.8 

Coefficients:
              Estimate Std. Error t value Pr(>|t|)    
(Intercept)  1.658e+02  3.332e+01   4.976 7.66e-07 ***
y_hat       -1.037e+00  8.703e-02 -11.913  < 2e-16 ***
I(y_hat^2)   8.043e-04  5.508e-05  14.601  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

Residual standard error: 662.9 on 986 degrees of freedom
Multiple R-squared:  0.1778,	Adjusted R-squared:  0.1761 
F-statistic: 106.6 on 2 and 986 DF,  p-value: < 2.2e-16
